#### Pre-processing

Cropping
- remove wanted protion

Resizing
- change dimension of image

In [ ]:
# transforming images
from transformers import image_transforms
from PIL import Image
import numpy as np

original_image = Image.open('image.jpg') #just imagine there's one
# Create the numpy array
image_array = np.array(original_image)

# Crop the center of the image
cropped_image = image_transforms.center_crop(image=image_array, size=(200, 200))

imgplot = plt.imshow(cropped_image)
plt.show()


# image classification
# Create the pipeline
image_classifier = pipeline(task="image-classification", 
            model="abhishek/autotrain_fashion_mnist_vit_base")

# Predict the class of the image
results = image_classifier(cropped_image)

# Print the results
print(results[0]["label"])

#### document/visual question & answering
Answering question based on a provided document, content can be text or image
visual: answering question based on provided image or video

Different from classifying & labeling, this involves answering questions

In [ ]:
# Create the pipeline
dqa = pipeline(task="document-question-answering", model="naver-clova-ix/donut-base-finetuned-docvqa")
vqa = pipeline(task="image-question-answering", model="vinai/bert-base-cased-vqa")  # what is this species in the picture?

# Set the image and question
image = "document.png"
question = "Which meeting is this document about?"

# Get the answer
results = dqa(image=image, question=question)

print(results)

#### Audio classification/Automatic speech recognition
Based on length and amplitude, sound wave can be converted to discrete values (digital representation) and then analyzed.
Speech models trained at 16hz

Resampling
- allows same sampling rate across all files to ensure consistency

Filtering
- ensuring enough sample size
- reduce computation by limiting file size
- Librosa library is for this task


In [ ]:
##  change sampling rate
# Save the old sampling rate
old_sampling_rate = audio_file[1]["audio"]["sampling_rate"]
# Resample the audio files
# change the sampling rate to 16000 Hz for column "audio"
audio_file = audio_file.cast_column("audio", Audio(sampling_rate=16_000))
# Compare the old and new sampling rates
print("Old sampling rate:", old_sampling_rate)
print("New sampling rate:", audio_file[1]["audio"]["sampling_rate"])


## Filter audios based on duration
# Create a list of durations
old_durations_list = []
# Loop over the dataset
for row in dataset["path"]:
    old_durations_list.append(librosa.get_duration(path=row))
# Create a new column
dataset = dataset.add_column("duration", old_durations_list)
# Filter the dataset
filtered_dataset = dataset.filter(lambda d: d < 6.0, input_columns=["duration"], keep_in_memory=True)


## audio classification
# Create the pipeline
classifier = pipeline(task="audio-classification", model="facebook/mms-lid-126")
# Extract the sample
audio = dataset[1]["audio"]["array"]  # extract np array that stores discreted audio
sentence = dataset[1]["sentence"]
# Predict the language
prediction = classifier(audio)
print(f"Predicted language is '{prediction[0]['label'].upper()}' for the sentence '{sentence}'")

#### Automatic Speech Recognition (ASR)
Use Word Error Rate (WER) to evaluate the ASR systems. Ranges to 0 - 1, the smaller the number, the more accurate the prediction.
Can be calculated using Hugging Face

In [ ]:
## asr with Open AI's whisper model
open_asr = pipeline(task="automatic-speech-recognition", model="openai/whisper-tiny")
open_pred = open_asr(example["audio"]["array"])["text"].lower()


## calculate WER
from evaluate import load
# Create the word error rate metric
wer = load("wer")
# Save the true sentence of the example
true_sentence = example["sentence"].lower()
open_wer = wer.compute(predictions=[open_pred], references=[true_sentence])



## Run asr for a larger number of audios
# Create the data function
def data(n=3):
    for i in range(n):
        yield english[i]["audio"]["array"], english[i]["sentence"].lower() 

# The data function uses lazy evaluation with the yield keyword. 
# Instead of loading all audio files into memory at once (which could take a lot of time and resources), it yields one audio file and its corresponding text at a time. 
# Ensures lower memory usage and less initialization time.

# Predict the text for the audio file with both models
output = []
for audio, sentence in data():
    meta_pred = meta_asr(audio)["text"].lower()
    open_pred = open_asr(audio)["text"].lower()
    # Append to output list
    output.append({"sentence": sentence, "metaPred": meta_pred, "openPred": open_pred})

output_df = pd.DataFrame(output)

# Compute the WER for both models
metaWER = wer.compute(predictions=output_df["metaPred"], references=output_df["sentence"])
openWER = wer.compute(predictions=output_df["openPred"], references=output_df["sentence"])

# Print the WER
print(f"The WER for the meta model is {metaWER} and for the open model is {openWER}")


Notes on the Generator Function with `yield`:
The yield keyword makes data a generator function. Unlike return, which exits the function and sends a value back to the caller, yield pauses the function's execution, saves its state, and allows it to resume from where it left off.

When the generator is iterated (e.g., using for audio, sentence in data()), yield provides the next set of values (audio and sentence) without restarting the function. This makes it well-suited for scenarios where processing data in chunks or streams is required, such as handling large datasets or audio files.

Can be interpreted as "the generator function is triggered in the for loop. When the for loop loops to a new set of values, the generator function generates the audio and sentence output"

- When the for loop starts (e.g., for audio, sentence in data()), the generator function data is called for the first time.
The generator starts executing, and when it encounters the first yield statement, it produces the first set of values (audio and sentence) and pauses.
Resuming the Generator:

- When the for loop moves to the next iteration, the generator resumes execution right after the last yield statement.
It continues until it reaches the next yield, where it produces the next set of values, and pauses again.